In [1]:
import sys
from libsbml import *
import re

In [62]:
def add_reaction(model, reactants, products, rx_type, pars):
    reaction_ids = [model.getReaction(i).getId() for i in range(model.getNumReactions())]
    reaction = model.createReaction()
    reaction.setFast(False)
    for par in pars.values():
        add_parameter(model, par)            
    for reactant in reactants:
        add_species(model, reactant)
        species = reaction.createReactant()
        species.setSpecies(reactant)
        species.setConstant(True)
    for product in products:
        add_species(model, product)        
        species = reaction.createProduct()
        species.setSpecies(product)
        species.setConstant(True)
    if rx_type=="binding":
        reaction_id = 'v'+'_'.join(reactants)
        if any(key not in pars.keys() for key in ['kon', 'koff']):
            raise ValueError("need both 'kon' and 'koff' parameters for binding reaction.")
        formula = " * ".join([pars["kon"]]+reactants)+" - "+" * ".join([pars["koff"]]+products)
    elif rx_type=="catalytic":
        reaction_id = 'v'+'_'.join(products)+'_cat'
        if 'kcat' not in pars.keys():
            raise ValueError("need 'kcat' parameter for catalytic reaction.")        
        formula = " * ".join([pars["kcat"]] + reactants)
    else:
        raise ValueError("Unknown reaction type.")
    if reaction_id in reaction_ids:
        raise ValueError("reaction "+reaction_id+" already exists in model.")        
    reaction.setId(reaction_id)
    math_ast = parseL3Formula(formula)
    kinetic_law = reaction.createKineticLaw()
    kinetic_law.setMath(math_ast)
    
def add_species(model, species_id, boundaryCondition = False):
    species_ids = [model.getSpecies(i).getId() for i in range(model.getNumSpecies())]
    if species_id not in species_ids:
        species = model.createSpecies()
        species.setId(species_id)
        species.setName(species_id)
        species.setCompartment('unnamed')
        species.setConstant(False)
        species.setInitialAmount(0.0)
        if boundaryCondition:
            species.setBoundaryCondition(boundaryCondition)

def add_parameter(model, par_id):
    parameter_ids = [model.getParameter(i).getId() for i in range(model.getNumParameters())]
    if par_id not in parameter_ids:    
        parameter = model.createParameter()
        parameter.setId(par_id)
        parameter.setConstant(False)
        parameter.setValue(1.0)
    
def add_to_cons(model, free_var, new_var):
    species_ids = [model.getSpecies(i).getId() for i in range(model.getNumSpecies())]
    if free_var not in species_ids:
        add_species(model, free_var, boundaryCondition = True)
    for var in [new_var]:
        if var not in species_ids:
            add_species(model, var)
    rule = model.getAssignmentRuleByVariable(free_var)
    if rule is None:
        tot_var = free_var + "tot"
        if tot_var not in species_ids:
            add_species(model, tot_var)
        rule = model.createAssignmentRule()
        rule.setVariable(free_var)
        rule.setFormula(free_var + "tot - " + new_var)
    else:
        formula = rule.getFormula()
        if '(' not in formula:
            formula.split(' ')
            components = formula.split(' - ')
            new_formula = components[0] + ' - ' + '(' + ' + '.join(components[1:] + [new_var]) + ')'
        else:
            formula = rule.getFormula()
            new_formula = formula[:-1]+' + '+new_var+')'
        rule.setFormula(new_formula)

def add_to_tot(model, tot_var, new_var):
    species_ids = [model.getSpecies(i).getId() for i in range(model.getNumSpecies())]
    for var in [tot_var, new_var]:
        if var not in species_ids:
            add_species(model, var)    
    rule = model.getAssignmentRuleByVariable(tot_var)
    if rule is None:
        rule = model.createAssignmentRule()
        rule.setVariable(tot_var)
        rule.setFormula(new_var)
    else:
        formula = rule.getFormula()    
        new_formula = formula+' + '+new_var
        rule.setFormula(new_formula)

### Add substrate that competes with ORF45

In [63]:
reader = SBMLReader()
document = reader.readSBML("../models/model_invitro_incell.xml")
model = document.getModel()

## get all rp species
pR_rule =  model.getAssignmentRule('pRtot')
formula = pR_rule.getFormula()
pR_species_list = formula.split(" + ")

## all rp species that can contribute to phosphorylation of the substrate
binding_species_list = [species for species in pR_species_list if "OpR" not in species and "P2" not in species]

## add reactions which contain substrate if it's already bound to pR
all_reactions = model.getListOfReactions()
all_species = model.getListOfSpecies()
for rx in all_reactions:
    products = [species for species in binding_species_list if rx.getProduct(species) is not None]
    if len(products) > 0:
        reactants = [species.getId() for species in all_species if rx.getReactant(species.getId()) is not None]
        if len(reactants) == 2:
            new_products = [product+"SUBC" for product in products]
            new_reactants = [re.sub("pR(.*)", "pR\\1SUBC", reactant) for reactant in reactants]
            formula = rx.getKineticLaw().getFormula()
            f_split = re.split(" ", formula)
            new_formula = " ".join([re.sub("pR(.*)", "pR\\1SUBC", s) if "k" not in s else s for s in f_split])
            rx_new = model.createReaction()
            rx_new.setId("v"+"_".join(new_reactants)+"__"+"_".join(new_products))
            for reactant in new_reactants:
                add_species(model, reactant)
                species = rx_new.createReactant()
                species.setSpecies(reactant)
                species.setConstant(True)                
            for product in new_products:
                add_species(model, product)
                species = rx_new.createProduct()
                species.setSpecies(product)
                species.setConstant(True)          
            new_law = rx_new.createKineticLaw()
            new_law.setFormula(new_formula)
        
## assemble reactions in which rp species binds to substrate
binding_reactions = [
    {'reactants': [pR_species, 'SUBC'],
     'products': [pR_species+'SUBC'],
     'bind_pars': {'kon': 'kon_pRSUB', 'koff': 'koff_pRSUB'}
    } for pR_species in binding_species_list
]

## assemble catalytic reactions
reactants_set = {d['products'][0] for d in binding_reactions}
catalytic_reactions = [
    {'reactants': [reactants],
     'products': [re.sub("SUBC", "", reactants), 'pSUBC'],
     'cat_pars': {'kcat': 'kp_SUB'}
    } for reactants in reactants_set
] + [
    {'reactants': ['pSUBC'],
     'products': ['SUBC'],
     'cat_pars': {'kcat': 'kdp_SUB'}
    }
]

add_to_cons(model, "SUBC", "pSUBC")



for species in {reaction['products'][0] for reaction in binding_reactions}:
    for free_species in ['R', 'O', 'E', 'SUBC']:
        if free_species in species:
            add_to_cons(model, free_species, species)
    for p_species in ['pR', 'pE']:
        if p_species in species:
            add_to_tot(model, p_species+'tot', species)

for reaction in binding_reactions:
    add_reaction(model, 
                 reaction['reactants'],
                 reaction['products'],
                 rx_type = "binding",
                 pars=reaction['bind_pars'])

for reaction in catalytic_reactions:
    add_reaction(model, 
             reaction['reactants'],
             reaction['products'],
             rx_type = "catalytic",
             pars=reaction['cat_pars'])

### Add generic substrate

In [64]:
## add reactions which contain substrate if it's already bound to rp
all_reactions = model.getListOfReactions()
all_species = model.getListOfSpecies()

## get all rp species
pR_rule =  model.getAssignmentRule('pRtot')
formula = pR_rule.getFormula()
pR_species_list = formula.split(" + ")

for rx in all_reactions:
    products = [species for species in pR_species_list if rx.getProduct(species) is not None]
    if len(products) > 0:
        reactants = [species.getId() for species in all_species if rx.getReactant(species.getId()) is not None]
        if len(reactants) == 2:
            new_products = [product+"SUBF" for product in products]
            new_reactants = [re.sub("pR(.*)", "pR\\1SUBF", reactant) for reactant in reactants]
            formula = rx.getKineticLaw().getFormula()
            f_split = re.split(" ", formula)
            new_formula = " ".join([re.sub("pR(.*)", "pR\\1SUBF", s) if "k" not in s else s for s in f_split])
            rx_new = model.createReaction()
            rx_new.setId("v"+"_".join(new_reactants)+"__"+"_".join(new_products))         
            for reactant in new_reactants:
                add_species(model, reactant)
                species = rx_new.createReactant()
                species.setSpecies(reactant)
                species.setConstant(True)                
            for product in new_products:
                add_species(model, product)
                species = rx_new.createProduct()
                species.setSpecies(product)
                species.setConstant(True)               
            new_law = rx_new.createKineticLaw()
            new_law.setFormula(new_formula)


## assemble binding reactions in which rp species binds to substrate
binding_reactions = [
    {'reactants': [pR_species, 'SUBF'],
     'products': [pR_species+'SUBF'],
     'bind_pars': {'kon': 'kon_pRSUB', 'koff': 'koff_pRSUB'}
    } for pR_species in pR_species_list
]

reactants_set = {d['products'][0] for d in binding_reactions}
catalytic_reactions = [
    {'reactants': [reactants],
     'products': [re.sub("SUBF", "", reactants), 'pSUBF'],
     'cat_pars': {'kcat': 'kp_SUB'}
    } for reactants in reactants_set
] + [
    {'reactants': ['pSUBF'],
     'products': ['SUBF'],
     'cat_pars': {'kcat': 'kdp_SUB'}
    }
]

add_to_cons(model, "SUBF", "pSUBF")

for species in {reaction['products'][0] for reaction in binding_reactions}:
    for free_species in ['R', 'O', 'E', 'SUBF', 'pK', 'P2']:
        if free_species in species:
            add_to_cons(model, free_species, species)
    if 'P' in species and not 'P2' in species:
        add_to_cons(model, 'P', species)
    for p_species in ['pR', 'pE']:
        if p_species in species:
            add_to_tot(model, p_species+'tot', species)

for reaction in binding_reactions:
    add_reaction(model, 
                 reaction['reactants'],
                 reaction['products'],
                 rx_type = "binding",
                 pars=reaction['bind_pars'])

for reaction in catalytic_reactions:
    add_reaction(model, 
             reaction['reactants'],
             reaction['products'],
             rx_type = "catalytic",
             pars=reaction['cat_pars'])
    
with open("../models/model_with_substrates.xml", "w") as f:
    f.write(writeSBMLToString(document))